In [161]:
with open("input.txt", "r") as f:
    lines = [x.split("\n") for x in f.read().split("\n\n")]

In [162]:
lines[:3]

[['31,88,35,24,46,48,95,42,18,43,71,32,92,62,97,63,50,2,60,58,74,66,15,87,57,34,14,3,54,93,75,22,45,10,56,12,83,30,8,76,1,78,82,39,98,37,19,26,81,64,55,41,16,4,72,5,52,80,84,67,21,86,23,91,0,68,36,13,44,20,69,40,90,96,27,77,38,49,94,47,9,65,28,59,79,6,29,61,53,11,17,73,99,25,89,51,7,33,85,70'],
 ['50 83  3 31 16',
  '47  9 94 10 86',
  '61 22 53 46 74',
  '77 41 79 55 62',
  '97 78 43 73 40'],
 ['99 96 20 35 21',
  '38 17 48 69 68',
  ' 9 51 32 52 11',
  '67  8 42 89 27',
  '39 62 66 72 43']]

In [165]:
draws = lines[0]
draws = [int(x) for x in draws[0].split(",")]
print("Draws :")
print(draws[:5])
print("\n")


boards= lines[1:]
boards = [[[int(val) for val in row.split(" ") if val!=""] for row in board if row!=""] for board in boards]
print("Boards (excerpt) :")
print(boards[:5])

Draws :
[31, 88, 35, 24, 46]


Boards (excerpt) :
[[[50, 83, 3, 31, 16], [47, 9, 94, 10, 86], [61, 22, 53, 46, 74], [77, 41, 79, 55, 62], [97, 78, 43, 73, 40]], [[99, 96, 20, 35, 21], [38, 17, 48, 69, 68], [9, 51, 32, 52, 11], [67, 8, 42, 89, 27], [39, 62, 66, 72, 43]], [[33, 16, 4, 78, 31], [96, 66, 13, 55, 18], [47, 89, 83, 99, 85], [50, 43, 39, 34, 98], [81, 65, 7, 23, 17]], [[24, 13, 57, 84, 50], [83, 86, 98, 92, 7], [28, 31, 85, 21, 12], [37, 48, 43, 47, 67], [19, 27, 1, 20, 16]], [[38, 75, 3, 14, 4], [8, 86, 98, 94, 83], [60, 46, 63, 85, 20], [69, 26, 73, 40, 29], [48, 84, 33, 18, 74]]]


In [158]:
assert all([len(row)==5  for board in boards for row in board ])

In [206]:
import numpy as np

class Board():
    def __init__(self, rows):
        self.board = np.array(rows)
        self.markers = np.zeros(self.board.shape)
        self.win = False
        
    def __str__(self):
        output = ""
        for i, row in enumerate(self.board):
            for j, val in enumerate(row):
                if self.markers[i,j] == 1:
                    item = "x"
                else:
                    item = val
                output += f"{item:<4}"
            output += "\n"
        return output
    
    def __repr__(self):
        return self.__str__()
    
    def lookup(self, value):
        rows, cols = np.where(self.board == value)
        if len(rows) != 0:
            for row, col in zip(rows, cols):
                self.mark((row, col))
    
    def mark(self, position:tuple):
        self.markers[position] = 1
    
    def check_win(self):
        self.win = any([sum(row)==5 for row in self.markers]) or any([sum(col)==5 for col in self.markers.transpose()])
        return self.win

In [246]:
class Bingo():
    def __init__(self, boards:list, draws:list):
        self.boards = [Board(board) for board in boards]
        self.winners = []
        self.draws = draws[::-1]
        self.current_draw = int()
        
        
    def start(self):
        while len(self.winners) == 0 and len(self.draws)>0:
            self.play_round()
        
        if len(self.winners)==0:
            print("There no winner for this game.")
            return 
        
        if len(self.winners)==1:
            print("And we have a winner!")
            self.announce_winner(self.winners[0])
        
        if len(self.winners)>1:
            # Shouldn't happen... 
            print("Several winners.. That's a tie!")
            for winner in self.winners:
                self.announce_winner(winner)
    
    def play_round(self):
        self.current_draw = self.draws.pop()
        for i, board in enumerate(self.boards):
            if not board.win:
                board.lookup(self.current_draw)
                board.check_win()
                if board.win:
                    print(f"Board n°{i} won")
                    self.winners.append(i)
            
    def announce_winner(self, board_id:int):
        score = self.get_score(board_id)
        print(f"Board {board_id} won with a score of {score} !\n")
        print(self.boards[board_id])
        
        
        
    def get_score(self, board_id):
        board = self.boards[board_id]
        assert board.win
        # sum of all unmarked numbers
        sum_unmarked = 0
        for i, row in enumerate(board.board):
            for j, col in enumerate(row):
                if board.markers[i,j] ==0:
                    sum_unmarked += board.board[i,j]
        
        # Multiply by the last draw
        score = sum_unmarked * self.current_draw
        
        return score

In [247]:
game = Bingo(boards, draws)

In [248]:
game.start()

Board n°66 won
And we have a winner!
Board 66 won with a score of 67716 !

84  78  3   44  96  
59  86  70  80  x   
93  x   52  x   61  
x   x   x   x   x   
5   25  6   85  99  



In [249]:
class BingoForLoosers(Bingo):
    def __init__(self, boards, draws):
        super().__init__(boards, draws)
    
    def start(self):
        while len(self.winners) != len(boards) and len(self.draws)>0:
            self.play_round()
            
        self.announce_looser(self.winners[-1])
        
    def announce_looser(self, board_id):
        score = self.get_score(board_id)
        print(f"Board {board_id} lost with a score of {score}..\n")
        print(self.boards[board_id])

In [250]:
game = BingoForLoosers(boards, draws)
game.start()

Board n°66 won
Board n°93 won
Board n°43 won
Board n°99 won
Board n°91 won
Board n°51 won
Board n°85 won
Board n°13 won
Board n°55 won
Board n°62 won
Board n°46 won
Board n°54 won
Board n°95 won
Board n°60 won
Board n°81 won
Board n°84 won
Board n°47 won
Board n°88 won
Board n°32 won
Board n°2 won
Board n°9 won
Board n°58 won
Board n°59 won
Board n°22 won
Board n°56 won
Board n°86 won
Board n°71 won
Board n°74 won
Board n°82 won
Board n°90 won
Board n°10 won
Board n°17 won
Board n°38 won
Board n°24 won
Board n°37 won
Board n°79 won
Board n°33 won
Board n°0 won
Board n°18 won
Board n°28 won
Board n°44 won
Board n°76 won
Board n°50 won
Board n°1 won
Board n°15 won
Board n°39 won
Board n°61 won
Board n°25 won
Board n°45 won
Board n°8 won
Board n°68 won
Board n°69 won
Board n°70 won
Board n°96 won
Board n°67 won
Board n°35 won
Board n°89 won
Board n°26 won
Board n°41 won
Board n°4 won
Board n°6 won
Board n°21 won
Board n°29 won
Board n°42 won
Board n°53 won
Board n°65 won
Board n°5 won
Boa